In [1]:
import numpy as np
from keras.layers import Dense, Activation
from keras.layers import SimpleRNN, LSTM, GRU
from keras.models import Sequential

In [2]:
# построчное чтение из примера с текстом 
with open("alice_in_wonderland.txt", 'rb') as _in:
    lines = []
    for line in _in:
        # удаляем лишние пробелы, проводим к нижнему регистру, убираем все лишние символы
        line = line.strip().lower().decode("ascii", "ignore")
        # удаляем пустые строки
        if len(line) == 0:
            continue
        lines.append(line)
text = " ".join(lines)
# разбиваем текст на последовательность символов и пропускаем через множетсво, чтобы остались только уникальные символы
chars = set([c for c in text])
nb_chars = len(chars)
nb_chars

55

получаем 55 символов: 26 букв + знаки препинания

In [3]:
# создание индекса символов и reverse mapping чтобы передвигаться между значениями numerical
# ID and a specific character. The numerical ID will correspond to a column
# ID и определенный символ. Numerical ID будет соответсвовать колонке
# число при использовании one-hot кодировки для представление входов символов
char2index = {c: i for i, c in enumerate(chars)}    # индекс смвола
index2char = {i: c for i, c in enumerate(chars)}    # символ индекса

In [4]:
# для удобства выберете фиксированную длину последовательность 10 символов 
SEQLEN, STEP = 10, 1
input_chars, label_chars = [], []

# конвертация data в серии разных SEQLEN-length субпоследовательностей
for i in range(0, len(text) - SEQLEN, STEP):
    input_chars.append(text[i: i + SEQLEN])
    label_chars.append(text[i + SEQLEN])

# Вычисление one-hot encoding входных последовательностей X и следующего символа (the label) y

X = np.zeros((len(input_chars), SEQLEN, nb_chars), dtype=bool)
y = np.zeros((len(input_chars), nb_chars), dtype=bool)
for i, input_char in enumerate(input_chars):
    for j, ch in enumerate(input_char):
        X[i, j, char2index[ch]] = 1
    y[i, char2index[label_chars[i]]] = 1


In [5]:
# установка ряда метапамертров  для нейронной сети и процесса тренировки
BATCH_SIZE, HIDDEN_SIZE = 128, 128
NUM_ITERATIONS = 15 # 25 должно быть достаточно
NUM_EPOCHS_PER_ITERATION = 1
NUM_PREDS_PER_EPOCH = 100

In [6]:
%%time

# Создайте суперпростую рекуррентную нейронную сеть. Имеется один рекуррентный
# слой, который производит вложение размера HIDDEN_SIZE из одноточечного
# кодированного входного слоя. За ним следует плотный слой с полной связью
# по набору возможных следующих символов, который преобразуется в
# вероятностную оценку с помощью стандартной активации softmax с мультиклассовой
# функцией потерь с перекрестной энтропией, связывающей предсказание с меткой одночасового
# меткой символа кодировки.


'''
Создание очень простой рекуррентной нейронной сети. В ней будет один реккурентный закодированный входной слой. За ним последует полносвязный слой связанный с набором возможных следующих символов, которые конвертированы в вероятностные результаты через стандартную softmax активацию с multi-class cross-encoding loss функцию ссылающуются на предсказание one-hot encoding лейбл символа
'''

model = Sequential()
model.add(
    GRU(  # вы можете изменить эту часть на LSTM или SimpleRNN, чтобы попробовать альтернативы
        HIDDEN_SIZE,
        return_sequences=False,
        input_shape=(SEQLEN, nb_chars),
        unroll=True
    )
)
model.add(Dense(nb_chars))
model.add(Activation("softmax"))
model.compile(loss="categorical_crossentropy", optimizer="rmsprop")


# выполнение серий тренировочных и демонстрационных итераций 
for iteration in range(NUM_ITERATIONS):

    # для каждой итерации запуск передачи данных в модель 
    print("=" * 50)
    print("Итерация #: %d" % (iteration))
    model.fit(X, y, batch_size=BATCH_SIZE, epochs=NUM_EPOCHS_PER_ITERATION)

    # Select a random example input sequence.
    test_idx = np.random.randint(len(input_chars))
    test_chars = input_chars[test_idx]

    # для числа шагов предсказаний использование текущей тренируемой модели 
    # конструирование one-hot encoding для тестирования input и добавление предсказания.
    print("Генерация из посева: %s" % (test_chars))
    print(test_chars, end="")
    
    for i in range(NUM_PREDS_PER_EPOCH):

        # здесь one-hot encoding.
        X_test = np.zeros((1, SEQLEN, nb_chars))
        for j, ch in enumerate(test_chars):
            X_test[0, j, char2index[ch]] = 1

        # осуществление предсказания с помощью текущей модели.
        pred = model.predict(X_test, verbose=0)[0]
        y_pred = index2char[np.argmax(pred)]

        # вывод предсказания добавленного к тестовому примеру 
        print(y_pred, end="")

        # инкрементация тестового примера содержащего предсказание
        test_chars = test_chars[1:] + y_pred
print()

Итерация #: 0
1241/1241 [==============================] - 20s 14ms/step - loss: 2.3535
Генерация из посева: ittle bat!
ittle bat! the said the said the said the said the said the said the said the said the said the said the said ==================================================
Итерация #: 1
1241/1241 [==============================] - 17s 14ms/step - loss: 1.9787
Генерация из посева: neezes: he
neezes: her and the was she was she was she was she was she was she was she was she was she was she was she wa==================================================
Итерация #: 2
1241/1241 [==============================] - 17s 13ms/step - loss: 1.8158
Генерация из посева: the cook. 
the cook. the the the the the the the the the the the the the the the the the the the the the the the the the ==================================================
Итерация #: 3
1241/1241 [==============================] - 17s 13ms/step - loss: 1.6993
Генерация из посева:  that prov
 that proved the was the was the was 

1:35:35 дописать двунаправленную сеть